In [127]:
import numpy as np
import pandas as pd
from transformers import pipeline
from PyPDF2 import PdfReader
import torch
import fitz
from pdfminer.high_level import extract_text
from pdfminer.layout import LTChar
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar,LTLine,LAParams
import re
import os

In [48]:
data_dir = "Venuratech\\flashcards_project\\data\\NEET PREVIOUS YEAR  (2019) - Sheet1.csv"
pdf_dir = "C:\\Users\\rishi\\OneDrive\\Desktop\\projects\\Venuratech\\ncert_pdf\\lebo102.pdf"
questions_dir = "C:\\Users\\rishi\\OneDrive\\Desktop\\projects\\Venuratech\\Flash-Card-Generator\\files\\questions.csv"
context_dir = "C:\\Users\\rishi\\OneDrive\\Desktop\\projects\\Venuratech\\Flash-Card-Generator\\files\\output.csv"

In [7]:
def get_answer(context, question):
    # Use the pre-trained model to get the answer
    qa_pipeline = pipeline('question-answering', model='deepset/roberta-base-squad2', \
                        tokenizer='deepset/roberta-base-squad2')
    result = qa_pipeline(context=context, question=question)
    answer = result['answer']
    return answer

In [24]:
questions_df = pd.read_csv(questions_dir)
contexts_df = pd.read_csv(context_dir)

In [39]:
questions = questions_df[questions_df['Page']==1]['Question'].values
questions

array(['What is the reproductive event in humans?',
       'What is the male reproductive system located in?',
       'Where is the male reproductive?',
       'What is the name of the sperms in males?',
       'What is the name of the text that includes a pair of testes along with accessory ducts glands?',
       'Where is the female reproductive system located?',
       'What are the differences between the reproductive events in the male and female?',
       'What is the female reproductive system?',
       'What is the term for transfer of sperms into the female?',
       'Where is the male reproductive system located?',
       'What is the female reproductive viviparous?',
       'What is the female reproductive system located in?',
       'What is the male reproductive system?',
       'What is the name of the sperms in females?',
       'What is the name of the genital tract insemination?',
       'What is the male reproductive viviparous?',
       'What does ovum cease to do in

In [40]:
context = contexts_df[context_df['Page']==1]["Preprocessed Text"].values[0]
context

'chapter 2 human reproduction 2.1 the male reproductive system as you are aware humans are sexually reproducing and 2.2 the female reproductive viviparous. the reproductive events in humans include system formation of gametes gametogenesis i.e. sperms in males 2.3 gametogenesis and ovum in females transfer of sperms into the female 2.4 menstrual cycle genital tract insemination and fusion of male and female gametes fertilisation leading to formation of zygote. this 2.5 fertilisation and is followed by formation and development of blastocyst implantation and its attachment to the uterine wall implantation 2.6 pregnancy and embryonic embryonic development gestation and delivery of the development baby parturition. you have learnt that these reproductive events occur after puberty. there are remarkable 2.7 parturition and lactation differences between the reproductive events in the male and in the female for example sperm formation continues even in old men but formation of ovum ceases in

In [34]:
pages = contexts_df['Page'].values
pages

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int64)

In [41]:
for question in questions:
    answer = get_answer(context,question)
    print(f"Q : {question}\nA : {answer}")

Q : What is the reproductive event in humans?
A : system formation of gametes gametogenesis
Q : What is the male reproductive system located in?
A : pelvis region
Q : Where is the male reproductive?
A : pelvis region
Q : What is the name of the sperms in males?
A : gametes gametogenesis
Q : What is the name of the text that includes a pair of testes along with accessory ducts glands?
A : 2.1a
Q : Where is the female reproductive system located?
A : 2.2 the female reproductive viviparous
Q : What are the differences between the reproductive events in the male and female?
A : 2.7 parturition and lactation
Q : What is the female reproductive system?
A : viviparous
Q : What is the term for transfer of sperms into the female?
A : ovum
Q : Where is the male reproductive system located?
A : pelvis region
Q : What is the female reproductive viviparous?
A : 2.2
Q : What is the female reproductive system located in?
A : viviparous
Q : What is the male reproductive system?
A : sexually reproducin

In [ ]:
def extract_text_by_font(pdf_path, target_font):
    doc = fitz.open(pdf_path)
    text = ""

    for page_number in range(doc.page_count):
        page = doc[page_number]
        block_list = page.get_text("blocks")

        for block in block_list:
            block_font = block[0]["font"]
            if block_font == target_font:
                text += block[4].strip() + "\n"

    doc.close()
    return text

In [66]:
doc = fitz.open(pdf_dir)

In [73]:
block_list = doc[1].get_text("blocks")


In [81]:
block_list[0].get("font",{})

AttributeError: 'tuple' object has no attribute 'get'

In [83]:
page = doc[1]
block_list = page.get_text("blocks")

for block in block_list:
    font_info = block[0].get("font", {})
    print(font_info)

AttributeError: 'float' object has no attribute 'get'

In [ ]:
# Font_size of the text is 10.5 keep in mind.So our traget_font_size =10.5

In [114]:
def seperate_text_by_font(path):
    Extract_Data=[]
    for page_layout in extract_pages(path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    for character in text_line:
                        if isinstance(character, LTChar):
                            Font_size=character.size
                Extract_Data.append([Font_size,(element.get_text())])
    return Extract_Data

In [115]:
context_list = seperate_text_by_font(pdf_dir)

In [117]:
context_list

[[26.0, 'CHAPTER 2\n'],
 [30.0, 'HUMAN REPRODUCTION\n'],
 [10.0, '2.1 The Male Reproductive\n'],
 [10.0, 'System\n'],
 [10.0, '2.2 The Female Reproductive\n'],
 [10.0, 'System\n'],
 [10.0, '2.3 Gametogenesis\n'],
 [10.0, '2.4 Menstrual Cycle\n'],
 [10.0, '2.5 Fertilisation and\nImplantation\n'],
 [10.0, '2.6 Pregnancy and Embryonic\n'],
 [10.0, 'Development\n'],
 [10.0, '2.7 Parturition and Lactation\n'],
 [10.5,
  'As you are aware, humans are sexually reproducing and\nviviparous. The reproductive events in humans include\nformation of gametes (gametogenesis), i.e., sperms in males\nand ovum in females, transfer of sperms into the female\ngenital tract (insemination) and fusion of male and female\ngametes (fertilisation) leading to formation of zygote. This\nis followed by formation and development of blastocyst\nand  its  attachment  to  the  uterine  wall  (implantation),\nembryonic development (gestation) and delivery of the\nbaby (parturition). You have learnt that these reproduct

In [148]:
def seperate_text_by_font(path):
    Extract_Data=[]
    for page_layout in extract_pages(path):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    for character in text_line:
                        if isinstance(character, LTChar):
                            Font_size=character.size
                Extract_Data.append([Font_size,(element.get_text())])
    return Extract_Data

def get_only_main_text(dir):
    main_text =[]
    context_list = seperate_text_by_font(dir)
    for context in context_list:
        if round(context[0],ndigits=4)== 10.5:
            main_text.append(context[1])
    return main_text

def clean_main_text (dir):
    pattern_to_remove = r'\(Figure\s+[^)]*\)'
    main_text_list = get_only_main_text(dir)
    modified_text_list =[]
    for text in main_text_list:
        modified_text = text.replace("\n", " ")
        processed_text = re.sub(pattern_to_remove, '', modified_text)
        modified_text_list.append(processed_text)
    return "\n".join(modified_text_list)
        
        



In [120]:
main_context = get_only_main_text(pdf_dir)

In [121]:
main_context

['As you are aware, humans are sexually reproducing and\nviviparous. The reproductive events in humans include\nformation of gametes (gametogenesis), i.e., sperms in males\nand ovum in females, transfer of sperms into the female\ngenital tract (insemination) and fusion of male and female\ngametes (fertilisation) leading to formation of zygote. This\nis followed by formation and development of blastocyst\nand  its  attachment  to  the  uterine  wall  (implantation),\nembryonic development (gestation) and delivery of the\nbaby (parturition). You have learnt that these reproductive\nevents  occur  after  puberty.  There  are  remarkable\ndifferences between the reproductive events in the male\nand in the female, for example, sperm formation continues\neven in old men, but formation of ovum ceases in women\naround the age of fifty years. Let us examine the male and\nfemale reproductive systems in human.\n',
 'The male reproductive system is located in the pelvis region\n(Figure  2.1a).  It

In [143]:
modified_string = main_context[6].replace("\n", " ")

In [144]:
modified_string

'The male accessory glands (Figure 2.1a, b) include paired seminal vesicles, a prostate and paired bulbourethral glands. Secretions of these glands constitute the seminal plasma which is rich in fructose, calcium and certain enzymes. The secretions of bulbourethral glands also helps in the lubrication of the penis. '

In [145]:
#pattern_to_remove = r"\(Figure\s+\d+(\.\d+[a-z])?(,\s*\d+(\.\d+[a-z])?)*\)"
pattern_to_remove = r'\(Figure\s+[^)]*\)'
modified_string_1 = re.sub(pattern_to_remove, '', modified_string)



In [146]:
modified_string_1

'The male accessory glands  include paired seminal vesicles, a prostate and paired bulbourethral glands. Secretions of these glands constitute the seminal plasma which is rich in fructose, calcium and certain enzymes. The secretions of bulbourethral glands also helps in the lubrication of the penis. '

In [149]:
processed_text = clean_main_text(pdf_dir)

In [151]:
print(processed_text)

As you are aware, humans are sexually reproducing and viviparous. The reproductive events in humans include formation of gametes (gametogenesis), i.e., sperms in males and ovum in females, transfer of sperms into the female genital tract (insemination) and fusion of male and female gametes (fertilisation) leading to formation of zygote. This is followed by formation and development of blastocyst and  its  attachment  to  the  uterine  wall  (implantation), embryonic development (gestation) and delivery of the baby (parturition). You have learnt that these reproductive events  occur  after  puberty.  There  are  remarkable differences between the reproductive events in the male and in the female, for example, sperm formation continues even in old men, but formation of ovum ceases in women around the age of fifty years. Let us examine the male and female reproductive systems in human. 
The male reproductive system is located in the pelvis region .  It  includes  a  pair  of  testes  alon

In [ ]:
text_file = open("processed_reproduction.txt", "w")
text_file.write(processed_text)
text_file.close()


In [13]:
import numpy as np
import pandas as pd

In [19]:
df = pd.DataFrame({
    0:np.zeros(10),
    1 : np.ones(10)
    })

In [20]:
df

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
5,0.0,1.0
6,0.0,1.0
7,0.0,1.0
8,0.0,1.0
9,0.0,1.0


In [25]:
questions_dir = "C:\\Users\\rishi\\OneDrive\\Desktop\\projects\\Venuratech\\Flash-Card-Generator\\files\\generated_questions.csv"
pd.read_csv(questions_dir).values.flatten()[:2]

array(['What is the inner cell mass called in human reproduction blastomeres?',
       'What is the process called that produces sperms?'], dtype=object)